# Fragility Curves

ERAD allows users to build custom fragility curves and pass to the simulator. If no user defined model is passed, default profiles defined in `erad.fragility_curves`.

## Building a custom fragility curve

Each fragility curve in ERAD is mapped to a spicific asset type. Supported asset types can be found in `here`. Each curve additionally maps to a probability function. A probability function requires two key pirces of information.

- **distribution**: Probability distribution fuction for the fragility curve
- **parameters**: Parameters associated with the probability distribution fucntion
    * The first paramater of the distribution has to be a ERAD quantity. These can be imported from `erad.quantities`.
    * Rest of the parameters should be simple floating point values

```{note}
The interface supports all distribution functions supported [here](https://docs.scipy.org/doc/scipy/reference/stats.html).
```

In [1]:
from erad.models.fragility_curve import FragilityCurve, ProbabilityFunction
from erad.quantities import Speed
from erad.enums import AssetTypes

fragility_curve = FragilityCurve(
    asset_type=AssetTypes.distribution_poles,
    prob_function=ProbabilityFunction(
        distribution="norm",
        parameters=[
            Speed(35, "cm/s"),
            3,
        ],
    ),
)
fragility_curve.pprint()

FragilityCurve(
    name='',
    asset_type=<AssetTypes.distribution_poles: 6>,
    prob_function=ProbabilityFunction(
        name='',
        distribution='norm',
        parameters=[<Quantity(35, 'centimeter / second')>, 3.0]
    )
)

An instance of the fragility model builds and returns probability distribution functions that are then used to calculate survival probability of assets. The function can be sampled using the `sample` method. 

```{note}
Sampled values returned are ERAD quantities.
```

In [2]:
prob_model = fragility_curve.prob_function.prob_model
prob_model.sample()

<Quantity(37.6563461, 'centimeter / second')>

Additionally, failure probability at a particular stress level can be calculated using the `probability` method.

In [3]:
prob_model.probability(Speed(10, "cm/s"))

np.float64(3.929873434851009e-17)

A complete fragility curve may be plotted sampling over and interval. See example below

In [ ]:
import plotly.express as px
import pandas as pd

wind_speed = range(0, 100, 1)
failure_porb = [prob_model.probability(Speed(i, "cm/s")) for i in wind_speed]


df = pd.DataFrame({"peak ground velocity [cm/s]": wind_speed, "failure probability": failure_porb})

fig = px.line(
    df,
    x="peak ground velocity [cm/s]",
    y="failure probability",
    title=f"Failure probability of {fragility_curve.asset_type.name}",
)
fig.show()

`HazardFragilityCurves` is a container for a set of fragililty curves that react to defined asset property. This set should contain a curve for each asset type define in your asset system. Here we add two curves one for peak ground velocity and a second fragility curve for peak ground acceleration.

In [5]:
from erad.models.fragility_curve import HazardFragilityCurves

curve_set_1 = HazardFragilityCurves(
    name="MY_CURVES",
    asset_state_param="peak_ground_velocity",
    curves=[
        fragility_curve,
    ],
)
curve_set_1.pprint()

HazardFragilityCurves(
    name='MY_CURVES',
    asset_state_param='peak_ground_velocity',
    curves=[
        FragilityCurve(
            name='',
            asset_type=<AssetTypes.distribution_poles: 6>,
            prob_function=ProbabilityFunction(
                name='',
                distribution='norm',
                parameters=[<Quantity(35, 'centimeter / second')>, 3.0]
            )
        )
    ]
)

In [6]:
curve_set_2 = HazardFragilityCurves(
    name="MY_CURVES",
    asset_state_param="peak_ground_acceleration",
    curves=[
        FragilityCurve(
            asset_type=AssetTypes.distribution_poles,
            prob_function=ProbabilityFunction(
                distribution="norm",
                parameters=[
                    Speed(5, "cm/s**2"),
                    3,
                ],
            ),
        ),
    ],
)

A completed set of curves can now be added to the `HazardSystem`. This ensure users ability to serialize hazard model along with fragility curvers used to run hazard analysis.

In [7]:
from erad.models.asset import Asset
from erad.runner import HarzardSimulator
from erad.systems.asset_system import AssetSystem
from erad.systems.hazard_system import HazardSystem

asset = Asset.example()
asset_system = AssetSystem(auto_add_composed_components=True)
asset_system.add_component(asset.example())
hazard_scenario = HarzardSimulator(asset_system=asset_system)
hazard_system = HazardSystem.earthquake_example()
hazard_system.add_components(*[curve_set_1, curve_set_2])

hazard_scenario.run(hazard_system=hazard_system, curve_set="MY_CURVES")

/opt/homebrew/Caskroom/miniconda/base/envs/erad/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:502: UserWarning:

Ellipsis is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.



## Default fragility curves

Default fragility curves can be imported from the ERAD library. Note users can build annd pass user defined fragility curves


In [ ]:
from erad.default_fragility_curves import DEFAULT_FRAGILTY_CURVES


for i, hazard_curves in enumerate(DEFAULT_FRAGILTY_CURVES):
    img = f"{hazard_curves.name}.html"
    hazard_curves.plot(img, 0, 80, 1000)

    # assert img.exists(), "Plotting failed, image not created."